In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import general_utils as Ugen
import matplotlib.pyplot as plt
import pandas as pd
import math
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob, os

In [3]:
##GET DATA
#GET FILES
main_dir=os.getcwd()
allFiles = glob.glob(main_dir.split('/Studies')[0]+'/toronto_parking_data'+ "/*.csv")

#LOAD TO DATAFRAME
list_ = []
for file_path in allFiles:#allFiles[-3:]:
    try:
        df = pd.read_csv(file_path,index_col=None,quoting=2) #,error_bad_lines=False) #quoting=2
        list_.append(df)
        print ('successfully loaded: '+ file_path)
    except Exception as err: 
        print ('NOT loaded: '+ file_path)
        print (str(err))
tpt_df = pd.concat(list_)
tpt_df.head()

successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2012.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2013.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_3.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_4.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_dat

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***78746,20120101,192,STAND SIGNED TRANSIT STOP,60,0,NR,355 PARKSIDE DR,NaN,NaN,ON
1,***31670,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,NR,220 KING ST W,NaN,NaN,ON
2,***47870,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,N/S,ELM ST,W/O,ELIZABETH ST,ON
3,***68489,20120101,8,STD VEH HWY PROHIB TIME/DAY,60,1,N/S,WALTON ST,E/O,BAY ST,ON
4,***98802,20120101,337,PARK - ON BOULEVARD,50,1,S/S,SHEPPARD AVE E,E/O,NEILSON AVE,ON


In [4]:
print (tpt_df.shape)
tpt_df.head()

(8448205, 11)


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***78746,20120101,192,STAND SIGNED TRANSIT STOP,60,0,NR,355 PARKSIDE DR,NaN,NaN,ON
1,***31670,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,NR,220 KING ST W,NaN,NaN,ON
2,***47870,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,N/S,ELM ST,W/O,ELIZABETH ST,ON
3,***68489,20120101,8,STD VEH HWY PROHIB TIME/DAY,60,1,N/S,WALTON ST,E/O,BAY ST,ON
4,***98802,20120101,337,PARK - ON BOULEVARD,50,1,S/S,SHEPPARD AVE E,E/O,NEILSON AVE,ON


In [5]:
tpt_df=tpt_df.drop(['tag_number_masked','location3','location4','province'],axis=1)
print (tpt_df.shape)
tpt_df.head()

(8448205, 7)


,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2
0,20120101,192,STAND SIGNED TRANSIT STOP,60,0,NR,355 PARKSIDE DR
1,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,NR,220 KING ST W
2,20120101,5,PARK HWY PROHIBED TIME/DAY,40,1,N/S,ELM ST
3,20120101,8,STD VEH HWY PROHIB TIME/DAY,60,1,N/S,WALTON ST
4,20120101,337,PARK - ON BOULEVARD,50,1,S/S,SHEPPARD AVE E


In [6]:
tpt_fh_df=tpt_df[tpt_df['infraction_code']==15]
tpt_fh_df.location2.value_counts()

##the following are the top grossing fire hydrants....
##multiply each number by $100 to get total revenue from 2012-2015

393 UNIVERSITY AVE      1802
99 ATLANTIC AVE         1100
33 ELMHURST AVE         1057
361 UNIVERSITY AVE      1036
112 MERTON ST            868
56 THE ESPLANADE         855
592 MARKHAM ST           771
6 SPRING GARDEN AVE      604
500 WELLINGTON ST W      596
5100 YONGE ST            591
17 LILLIAN ST            577
5460 YONGE ST            573
5453 YONGE ST            552
5519 YONGE ST            538
21 GLEBE RD E            522
26 WELLINGTON ST E       519
DORIS AVE                478
90 KIPPENDAVIE AVE       471
25 BELLEVUE AVE          441
295 RONCESVALLES AVE     438
43 ELM ST                428
387 QUEEN ST W           418
44 BALLIOL ST            406
5645 YONGE ST            399
99 ERSKINE AVE           388
5150 YONGE ST            387
17 NIAGARA ST            383
150 KILGOUR RD           376
5533 YONGE ST            376
123 SUNNYSIDE AVE        373
                        ... 
471 DAWES RD               1
HOWARD ST                  1
2254 BLOOR ST W            1
1691 GERRARD S

In [99]:
##LETS FIND THE HIGHEST GROSSING PERIOD
infraction_ids=list(tpt_df.infraction_code.unique())

# tpt_df.groupby(['infraction_code','location2']).size()
#tpt_df.groupby(['infraction_code','location2']).location2.count()

tpt_df_mod=tpt_df.iloc[:500000]
tpt_df_mod=tpt_df_mod.sort_values('location2')

# tpt_df_mod['counts']=tpt_df_mod.groupby(['infraction_code','location2']).location2.transform('count')
# tpt_df_mod

In [100]:
tpt_df_mod.shape
# sorted_df.columns.values

(500000, 7)

In [101]:
#DROP MISSING LOCATIONS
tpt_df_mod=tpt_df_mod[pd.notnull(tpt_df_mod['location2'])]
tpt_df_mod.shape

(499746, 7)

In [113]:
new_df=pd.DataFrame({'count' : tpt_df_mod.groupby( ['infraction_code','location2'] ).size()}).reset_index()
print(new_df.shape)



(123593, 3)


In [132]:
##now calculate total amount...
##create dict of infraction_code:fine_amount
fine_amount_dict={ic:{'fine_amt':tpt_df[tpt_df['infraction_code']==ic].set_fine_amount.median(),
                      'fine_descp':tpt_df[tpt_df['infraction_code']==ic].infraction_description.iloc[0]}    
                          for ic in infraction_ids}


def calc_fine_sum(df):
    return df['count']*fine_amount_dict[df['infraction_code']]['fine_amt']

def get_fine_descrp(df):
    return fine_amount_dict[df['infraction_code']]['fine_descp']

new_df['fine_sum']=new_df.apply(calc_fine_sum,axis=1)
new_df['fine_descrp']=new_df.apply(get_fine_descrp,axis=1)

In [133]:
new_df.sort_values('fine_sum',ascending=False)

,infraction_code,location2,count,fine_sum,fine_descrp
10477,3,1750 FINCH AVE E,2190,65700,PARK/LEAVE ON PRIVATE PROPERTY
122969,367,410 COLLEGE ST,115,51750,STAND ON-ST DISABL LDG NO PRMT
121805,347,3401 DUFFERIN ST,132,33000,PARK IN A FIRE ROUTE
21607,5,20 EDWARD ST,821,32840,PARK HWY PROHIBED TIME/DAY
11128,3,2075 BAYVIEW AVE,1040,31200,PARK/LEAVE ON PRIVATE PROPERTY
122511,355,555 REXDALE BLVD,62,27900,PARK IN DISABLED NO PERMIT
14985,3,941 PROGRESS AVE,916,27480,PARK/LEAVE ON PRIVATE PROPERTY
121431,347,1800 SHEPPARD AVE E,107,26750,PARK IN A FIRE ROUTE
122934,367,18 GRENVILLE ST,56,25200,STAND ON-ST DISABL LDG NO PRMT
121926,347,45 OVERLEA BLVD,98,24500,PARK IN A FIRE ROUTE
